# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages.

In [1]:
!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
!gzip -d cc.ru.300.vec.gz

!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz
!gzip -d cc.uk.300.vec.gz

--2022-04-13 12:36:26--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ru.300.vec.gz
Распознаётся dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)… 188.114.96.128
Подключение к dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|188.114.96.128|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1306357571 (1,2G) [binary/octet-stream]
Сохранение в: «cc.ru.300.vec.gz»

cc.ru.300.vec.gz    100%[===================>]   1,22G  8,21MB/s    за 2m 34s  

2022-04-13 12:39:01 (8,08 MB/s) - «cc.ru.300.vec.gz» сохранён [1306357571/1306357571]

--2022-04-13 12:39:32--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.uk.300.vec.gz
Распознаётся dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)… 188.114.96.128
Подключение к dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|188.114.96.128|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1257595219 (1,2G) [binary/octet-stream]
Сохранение в: «cc.uk.300.vec.gz»

cc.uk.300.ve

After downloading and extracting the vectors, we should be able to load them using the [gensim](https://radimrehurek.com/gensim/) library:

In [2]:
from gensim.models import KeyedVectors


uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")
ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")

Once you've loaded the vectors, you can use the `KeyedVectors` interface to get word embeddings and/or query most similar words by embedding:

In [3]:
august_embedding = ru_emb["август"]
august_embedding.shape, august_embedding[:5]

((300,), array([ 0.0033, -0.0322, -0.0519, -0.0808, -0.0131], dtype=float32))

In [4]:
ru_emb.most_similar([august_embedding])

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262),
 ('июнь', 0.9222574830055237),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930036425590515),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545795679092407),
 ('февраль', 0.8401415944099426)]

The latter function also allows you to vary the amount of closest words via the `topn` argument:

In [5]:
ru_emb.most_similar([august_embedding], topn=3)

[('август', 1.0000001192092896),
 ('июль', 0.9383152723312378),
 ('сентябрь', 0.9240029454231262)]

Another feature of `KeyedVectors` is that it allows to compute embeddings for multiple words simultaneously:

In [8]:
ru_emb[["август", "сентябрь"]].shape

(2, 300)

Everything above is true for the embeddings for Ukrainian language.

In [9]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 0.9999998807907104),
 ('липень', 0.9096441268920898),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992518782615662),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633180618286),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.840811014175415),
 ('лютий', 0.8256431221961975)]

However, russian and ukrainian embeddings were trained independently of each other. This means, that there is no obvious connection between values in embeddings for similar words in Russian and Ukrainian:

In [10]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962644100189),
 ('ЖИЗНИВадим', 0.25203436613082886),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829229712486267),
 ('Ведущий-Алексей', 0.24438698589801788),
 ('Недопустимость', 0.24435284733772278),
 ('2Михаил', 0.23981398344039917),
 ('лексей', 0.23740755021572113),
 ('комплексн', 0.23695150017738342),
 ('персональ', 0.2368222326040268)]

## Translation

We'll build a simple translator, which will try to predict the russian embedding from the ukrainian one. For this we'll need a dataset of word pairs.

In [12]:
import pandas as pd


#train_data_url = "https://raw.githubusercontent.com/girafe-ai/ml-mipt/master/homeworks_advanced/Lab1_NLP/uk_ru.train.tsv"
train_data_url = 'uk_ru.train.tsv'
train_data = pd.read_csv(train_data_url, sep="\t", header=None)
train_data.columns = ["uk", "ru"]
print(f"Train dataset size: {len(train_data)}")

#test_data_url = "https://raw.githubusercontent.com/girafe-ai/ml-mipt/master/homeworks_advanced/Lab1_NLP/uk_ru.test.tsv"
test_data_url = 'uk_ru.test.tsv'
test_data = pd.read_csv(test_data_url, sep="\t", header=None)
test_data.columns = ["uk", "ru"]
print(f"Test dataset size: {len(test_data)}")

train_data.head()

Train dataset size: 1927
Test dataset size: 400


,uk,ru
0,iмовірно,вероятно
1,iснує,существует
2,iспит,экзамен
3,абияк,как-нибудь
4,або,или


Our method won't work with unknown words, so let's filter them out:

In [13]:
rows = []
for _, row in train_data.iterrows():
    if row["uk"] not in uk_emb or row["ru"] not in ru_emb:
        continue

    rows.append(row)

train_data = pd.DataFrame(rows)
print(f"Train dataset size: {len(train_data)}")

rows = []
for _, row in test_data.iterrows():
    if row["uk"] not in uk_emb or row["ru"] not in ru_emb:
        continue

    rows.append(row)

test_data = pd.DataFrame(rows)
print(f"Test dataset size: {len(test_data)}")

Train dataset size: 1880
Test dataset size: 393


We will train our model to predict embedding for the russian word from embedding of its ukrainian counterpart. For this reason we split our train and test data into ukrainian and russian words and compute corresponding embeddings to obtain `X` (ukrainian embeddings) and `y` (russian embeddings).

In [14]:
X_train, Y_train = uk_emb[train_data["uk"].values], ru_emb[train_data["ru"].values]
X_test, Y_test = uk_emb[test_data["uk"].values], ru_emb[test_data["ru"].values]

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$$

or

$$W^*= \arg\min_W \|XW^T - Y\|_F$$

where $\|\cdot\|_F$ denotes Frobenius norm.

> **Note:** in second formula, $W$ and $x$ seem to have switched places. This happens because the $X$ matrix is composed of objects $x_i$ in *rows* not *columns*, i.e. it is kind of composed of $x_i^T$. This means that $X \in \mathbb{R}^{N \times D}$, where $N$ is the number of items and $D$ is the embedding dimensionality. The same is true for the $Y$.

$W^*= \arg\min_W \sum_{i=1}^n\|Wx_i - y_i\|_2$ looks like simple multiple linear regression without bias. The `sklearn` allows you to turn off the bias in `LinearRegression` via the `fit_intercept` argument (in fact they simply call bias the intercept). So let's code.

In [17]:
from sklearn.linear_model import LinearRegression


# YOUR CODE HERE
mapping = LinearRegression(fit_intercept=False);
mapping.fit(X_train, Y_train);

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [19]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

[('апрель', 0.8531433343887329),
 ('июнь', 0.8402522206306458),
 ('март', 0.8385883569717407),
 ('сентябрь', 0.8331484198570251),
 ('февраль', 0.8311208486557007),
 ('октябрь', 0.8278018832206726),
 ('ноябрь', 0.8243728876113892),
 ('июль', 0.822961688041687),
 ('август', 0.8112279772758484),
 ('январь', 0.8022984862327576)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in russian embedding space).

In [22]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    total = len(pairs)
    correct = 0
    for i in range(total):
        pair = pairs[i]
        predicted_vector = mapped_vectors[i]
        ru_emb_most_similar = ru_emb.most_similar([predicted_vector], topn=topn)
        pred_ru_words, _ = zip(*ru_emb_most_similar)
        if pair[1] in pred_ru_words[: topn]:
            correct += 1

        # YOUR CODE HERE

    return correct / total

In [30]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

Note that our `precision` function accepts lists of pairs of words, whereas we have dataframes. However, it is not a problem: we can get a list (actually, numpy array) of pairs via the `values` property.

In [31]:
assert precision(test_data.values, X_test) == 0.0
assert precision(test_data.values, Y_test) == 1.0

Let's see how well our model is doing.

In [32]:
top1 = precision(test_data.values, mapping.predict(X_test), 1)
print(f"Top-1 precision {100 * top1:.1f}%")

top5 = precision(test_data.values, mapping.predict(X_test), 5)
print(f"Top-5 precision {100 * top5:.1f}%")

Top-1 precision 62.8%
Top-5 precision 79.1%


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$(W^T)^*= \arg\min_{W^T} \|XW^T - Y\|_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$(W^T)^*=UV^T$$

In [33]:
import numpy as np


# YOUR CODE HERE
# Compute the orthogonal mapping (W^T)^* as defined in formula above.
# mapping = ...
U, sigma, V_T = np.linalg.svd(np.dot(X_train.T, Y_train))
mapping = np.dot(U, V_T)

Now our `mapping` is just a numpy array, meaning that it has no `predict` method. However, from the formulae above we know, that prediction is done using the matrix multiplication:

In [285]:
august = uk_emb["серпень"] @ mapping
ru_emb.most_similar([august])

[('апрель', 0.8245132565498352),
 ('июнь', 0.8056629300117493),
 ('сентябрь', 0.8055761456489563),
 ('март', 0.8032934665679932),
 ('октябрь', 0.798710286617279),
 ('июль', 0.7946797013282776),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938189506530762),
 ('февраль', 0.7923861742019653),
 ('декабрь', 0.7715376019477844)]

Now let's compute our precision values and see, whether our trick did improve the results.

In [35]:
top1 = precision(test_data.values, X_test @ mapping, 1)
print(f"Top-1 precision {100 * top1:.1f}%")

top5 = precision(test_data.values, X_test @ mapping, 5)
print(f"Top-5 precision {100 * top5:.1f}%")

Top-1 precision 64.4%
Top-5 precision 79.9%


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [36]:
!wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
!gzip -d ./uk.txt.gz

--2022-04-13 13:58:06--  https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
Распознаётся object.pouta.csc.fi (object.pouta.csc.fi)… 86.50.254.18, 86.50.254.19
Подключение к object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1819128 (1,7M) [application/gzip]
Сохранение в: «uk.txt.gz»

uk.txt.gz           100%[===================>]   1,73M  1,31MB/s    за 1,3s    

2022-04-13 13:58:08 (1,31 MB/s) - «uk.txt.gz» сохранён [1819128/1819128]



In [37]:
with open('./uk.txt', 'r') as f:
    uk_corpus = f.readlines()

# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]
uk_corpus[:5]

['Я вже закінчу коледж, коли ви вернетеся з Америки.\n',
 'Він наказав мені негайно вийти з кімнати.\n',
 'Як би ти не намагався, ти не вивчиш англійську за два-три місяці.\n',
 'Поки я не подзвонив, він не прийшов.\n',
 'У всесвіті багато галактик.\n']

Now let's translate these sentences word-by-word. Before that, however, don't forget to tokenize your sentences. For that you may (or may not) find the `nltk.tokenize.WordPunctTokenizer` to be very useful.

In [38]:
from nltk.tokenize import WordPunctTokenizer

Слова OOV будем заменять на токен UNK

In [48]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    translated = []

    # YOUR CODE HERE
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(sentence)
    UNK = 'UNK'
    
    for token in tokens:
        if token in uk_emb:
            prediction = uk_emb[token] @ mapping
            rus_token, rus_embedding = ru_emb.most_similar([prediction], topn=1)[0]
            translated.append(rus_token)
        else:
            translated.append(UNK)
    return " ".join(translated)

In [49]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [73]:
for i, sent in enumerate(uk_corpus[::10]):
    print(i, translate(sent))

0 Я уже закончу колледж , когда мы прибежишь со Америки .
1 Город бомбили враждебные самолеты .
2 Возможно , мной антисоциальный , конечно это не означает , что мной не общаюсь со людьми .
3 Впрочем утра выпала роса .
4 Беда не приходит одна .
5 Посмотри по тот дым .
6 Я заказал два гамбургера .
7 Я не хотел никого обидеть .
8 Гора покрыта снегом .
9 по фотографии во девушки корона не со золота , а со цветов .
10 Во меня То мечта .
11 Я приехал во Японию со Китая .
12 по север находится Шотландия ; по юге — Англия ; по востоке — Уэльс ; и ещe дальше по востоке — северная Ирландия .
13 Его родная страна — Германия .
14 Берн — столица Швейцарии .
15 Он ждал по него к десятой часа .
16 Ты можешь взять ту книгу даром .
17 Такой роман сочинил известный американский писатель .
18 забронировать , будте ласковые , комнату возле международного аэропорта во Торонто .
19 Он знает , что ты его влюбится ?
20 Я знаю , что ты богатый .
21 Те , кто всё забывают , счастливые .
22 Во этой реке опасно пл

Посмотрим на те предложения, где есть OOV токен

In [77]:
for i, sent in enumerate(uk_corpus[::10]):
    if i in [35, 37, 47, 50, 60, 89, 94]:
        print(i, translate(sent))

35 поверхность воздушной шары — UNK пространство , потому для неё не выполняются правила симметрической геометрии .
37 Можно мной UNK это платье ?
47 UNK загрязнение можно было бы UNK только если бы люди были более чувствительны к окружающей среды .
50 Я бы со удовольствием сочинил сотни сложноподчинённые во UNK со и , конечно во меня То дела .
60 Со UNK Алексея Палашка поприветствовал президент Белоруссии Александр Лукашенко .
89 « Сколько стоить та носовая косыночка UNK — « тринадцать со двадцать аш со пять центов UNK
94 « Спасибо UNK — « по здоровье со мной UNK


Попробуем побороться с проблемой неизвестного токена. Воспользуемся идеей fasttext. Представим неизвестное слово как среднее эмбедингов его букв, умноженное на коэффициент (по умолчанию = 0.5)

In [116]:
def get_letters_embedding_oov(unk_token, koef=0.5):
    '''Эмбединг слова -- сумма эмбеддингов его букв'''
    emb = uk_emb[unk_token[0]].copy()
    for letter in unk_token[1: ]:
        emb += uk_emb[letter]
    return emb /  len(unk_token) * koef

In [117]:
def translate_letters(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    translated = []

    # YOUR CODE HERE
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(sentence)
    
    for token in tokens:
        if token not in uk_emb:
            emb = get_letters_embedding_oov(token)
        else:
            emb = uk_emb[token]
        prediction = emb @ mapping
        rus_token, rus_embedding = ru_emb.most_similar([prediction], topn=1)[0]
        translated.append(rus_token)
            
    

    return " ".join(translated)

In [118]:
for i, sent in enumerate(uk_corpus[::10]):
    if i in [35, 37, 47, 50, 60, 89, 94]:
        print(i, translate_letters(sent))

35 поверхность воздушной шары — ен пространство , потому для неё не выполняются правила симметрической геометрии .
37 Можно мной ва это платье ?
47 Во загрязнение можно было бы ой только если бы люди были более чувствительны к окружающей среды .
50 Я бы со удовольствием сочинил сотни сложноподчинённые во .n со и , конечно во меня То дела .
60 Со ݀ Алексея Палашка поприветствовал президент Белоруссии Александр Лукашенко .
89 « Сколько стоить та носовая косыночка ? — « тринадцать со двадцать аш со пять центов »
94 « Спасибо , — « по здоровье со мной »


Не особо это помогло.

Попробуем жадный алгоритм разбиения слова на известные непересекающиеся n-граммы

In [210]:
def get_ngramm_embedding_oov(unk_token):
    '''Эмбединг слова -- сумма эмбеддингов его непересекающихся n-грамм'''
    begin = 0
    end = 1
    parts = []
    while end <= len(unk_token):
        if unk_token[begin: end] in uk_emb:
            end += 1
        else:
            parts.append(unk_token[begin: end - 1])
            begin = end - 1
            end = begin + 1
    parts.append(unk_token[begin: ])
    
    emb = uk_emb[parts[0]].copy()
    for part in parts[1: ]:
        emb += uk_emb[part]
    return emb 

In [211]:
def translate_ngramm(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    translated = []

    # YOUR CODE HERE
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(sentence)
    
    for token in tokens:
        if token not in uk_emb:
            emb = get_ngramm_embedding_oov(token)
        else:
            emb = uk_emb[token]
        prediction = emb @ mapping
        rus_token, rus_embedding = ru_emb.most_similar([prediction], topn=1)[0]
        translated.append(rus_token)
    return " ".join(translated)

In [212]:
for i, sent in enumerate(uk_corpus[::10]):
    if i in [35, 37, 47, 50, 60, 89, 94]:
        print(i, translate_ngramm(sent))

35 поверхность воздушной шары — ой пространство , потому для неё не выполняются правила симметрической геометрии .
37 Можно мной еем это платье ?
47 ен загрязнение можно было бы ына только если бы люди были более чувствительны к окружающей среды .
50 Я бы со удовольствием сочинил сотни сложноподчинённые во jg со и , конечно во меня То дела .
60 Со еем Алексея Палашка поприветствовал президент Белоруссии Александр Лукашенко .
89 « Сколько стоить та носовая косыночка ? — « тринадцать со двадцать аш со пять центов »
94 « Спасибо , — « по здоровье со мной »


К сожалению, не удалось достичь приемлемых результатов.

Попробуем приблизить слово всеми n-граммами, которые есть в словаре

In [281]:
def get_ngramm_embedding_oov_2(unk_token):
    '''Эмбединг слова -- сумма эмбеддингов всех его n-грамм'''
    begin = 0
    end = 1
    parts = []
    while end <= len(unk_token):
        if unk_token[begin: end] in uk_emb:
            parts.append(unk_token[begin: end])
            end += 1
        else:
            begin += 1
            end = begin + 1
    #parts.append(unk_token[begin: ])
    #print(parts)
    emb = uk_emb[parts[0]].copy()
    for part in parts[1: ]:
        emb += uk_emb[part]
    return emb

In [283]:
def translate_ngramm_2(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    translated = []

    # YOUR CODE HERE
    tokenizer = WordPunctTokenizer()
    tokens = tokenizer.tokenize(sentence)
    
    for token in tokens:
        if token not in uk_emb:
            emb = get_ngramm_embedding_oov_2(token)
        else:
            emb = uk_emb[token]
        prediction = emb @ mapping
        rus_token, rus_embedding = ru_emb.most_similar([prediction], topn=1)[0]
        translated.append(rus_token)
    return " ".join(translated)

In [284]:
for i, sent in enumerate(uk_corpus[::10]):
    if i in [35, 37, 47, 50, 60, 89, 94]:
        print(i, translate_ngramm_2(sent))

35 поверхность воздушной шары — ен пространство , потому для неё не выполняются правила симметрической геометрии .
37 Можно мной ва это платье ?
47 ен загрязнение можно было бы ьс только если бы люди были более чувствительны к окружающей среды .
50 Я бы со удовольствием сочинил сотни сложноподчинённые во .n со и , конечно во меня То дела .
60 Со ен Алексея Палашка поприветствовал президент Белоруссии Александр Лукашенко .
89 « Сколько стоить та носовая косыночка ? — « тринадцать со двадцать аш со пять центов »
94 « Спасибо , — « по здоровье со мной »


И эта идея не привела к успеху :(

Great! 
See second notebook for the Neural Machine Translation assignment.